# Preprocessing

Before starting the analysis part of the project, a little preprocessing has to be done. To start, we create a list with all the stories in it. Next we take apart each token to save only the lemmatized token, and remove unnecessary tokens (such as "/n" or punctuation). 

In [9]:
import os
import glob

# appends to file_list all files inside the particular folder given below that end with ".txt"
file_list = []
for file_name in glob.iglob("data/**/*.txt", recursive=True):
    file_list.append(file_name)

# appends each line of each story to story_list
story_list = []
for file in file_list:
    with open(file, "r", encoding='UTF-8') as story:
        lines = story.readlines()
        story_list.append(lines)  

In [14]:
file_list[:5]

['data/cluster083/253635663-THE-FARMYARD-COCK-AND-THE-WEATHERCOCK-H-C-Andersen.txt',
 'data/cluster083/384500260-THE-BUTTERFLY-H-C-Andersen.txt',
 'data/cluster083/798449202-The-Three-Languages-Brothers-Grimm.txt',
 'data/cluster083/477325002-TWENTY-THIRD-EVENING-H-C-Andersen.txt',
 'data/cluster083/c235.txt']

In [26]:
story_list[0][:10]

['<title>/IN --> IN/IN\n',
 'THE/DT --> the/DT\n',
 'FARMYARD/NNP --> FARMYARD/NNP\n',
 'COCK/NNP --> COCK/NNP\n',
 'AND/CC --> and/CC\n',
 'THE/DT --> the/DT\n',
 'WEATHERCOCK/NNP --> WEATHERCOCK/NNP\n',
 './. --> ./.\n',
 '</title>/NNP --> NNP/NNP\n',
 '<author>/NNP --> NNP/NNP\n']

In [27]:
# removes the first, not letmmatized, instance of each token
trimmed_story_list = []
for sentence in story_list:
    for token in sentence:
        if "-->" in token:
            split_token = token.split("-->", 1)
            trimmed_story_list.append(split_token[1])
        else:
            continue

In [30]:
trimmed_story_list[0]

' IN/IN\n'

In [31]:
# removes "/n" from the end of each token
cleaned_list = []
for item in trimmed_story_list:
    new_item = item[:-1]
    cleaned_list.append(new_item)

In [33]:
cleaned_list[:10]

[' IN/IN',
 ' the/DT',
 ' FARMYARD/NNP',
 ' COCK/NNP',
 ' and/CC',
 ' the/DT',
 ' WEATHERCOCK/NNP',
 ' ./.',
 ' NNP/NNP',
 ' NNP/NNP']

In [34]:
# adds only relevant tokens to filtered_list: removes tokens that are not word, as well as words irrelevant to our 
# study (such as determinants)
filtered_list=[]
for item in cleaned_list:
    if item.startswith((" .", " ,", " /", " :", " ;", " '", " `", " NNP", " ?", " !", " -")): #numbers? cd
        continue
    elif item.endswith(("CC", "IN", "DT")):
        continue
    else:
        filtered_list.append(item)

In [35]:
filtered_list[:10]


[' FARMYARD/NNP',
 ' COCK/NNP',
 ' WEATHERCOCK/NNP',
 ' H./NNP',
 ' C./NNP',
 ' Andersen/NNP',
 ' there/EX',
 ' be/VBD',
 ' two/CD',
 ' cock/NNS']

# Data Exploration

Now we can move on the explore the data. To do so, we'll take a look at the stories themselves and what we can tell from the data before training any model on it.